# Project: Conversational AI Chatbot

Description:
Developed an advanced conversational chatbot for a customer service application using OpenAI APIs and LangChain.

Features:
1. Integrated memory for personalized responses.
2. Leveraged LangChain for document-based question answering.
3. Connected the bot to a vector database (Pinecone) for semantic search across support documents.





In [4]:
# !pip install langchain_community

In [33]:
import os
from langchain import vectorstores
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document
from pinecone import Pinecone, ServerlessSpec,Index


In [17]:
# Initialize OpenAI API Key
openai_api_key = os.getenv("OPENAI_API_KEY") 

# Initialize Pinecone API Key and Environment
pinecone_api_key = os.getenv("PINECONE_API_KEY") 
pinecone_env = "us-east1-gcp"  

In [21]:
# Initialize Pinecone Client
pinecone_client = Pinecone(api_key=pinecone_api_key)

# Create Pinecone Index if it doesn't exist
index_name = "chatbot-support-docs"
if index_name not in [idx.name for idx in pinecone_client.list_indexes()]:
    pinecone_client.create_index(
        name=index_name,
        dimension=1536,  # OpenAI embeddings dimension
        metric="cosine",
        spec=ServerlessSpec(cloud="aws",region="us-east-1"),
    )
             


In [25]:
# Retrieve index host
index_description = pinecone_client.describe_index(index_name)
host = index_description.host

In [26]:
# Connect to the Pinecone Index
index = Index(index_name, host=host)

In [27]:
# Initialize OpenAI Embeddings
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)


In [44]:
documents = [
    {"page_content": "Terrain Trait is a company started in the year 2089 by JJ Kiddo. JJ Kiddo was a lawyer working in Mar's BleevFlemin."},
    {"page_content": "Terrain Trait provides transportation support from Earth to Moon, Mars, and even for some cities in Andromeda."},
    {"page_content":  "Terrain Trait's wormhole technology was once sued for an alleged mafia case."},
    {"page_content": "Junior Juma Kiddo was a national-level chess player in Mandeep, Andromeda."},
    {"page_content": "Terrain Trait generated revenue of USD 560 Million in the last month by its Telecommunication division which provides communication services between different planets."},
    {"page_content": '''Places where transportation is offered: 
                            Milkyway : { Moon : [Bisle, Bubule, Nuansilei, PortBharat, Multh, Jadeo],
                                            Mars : [Rappos Valley, Kirkush, Meyo Prag, Jujunsi],
                                            Bhoo-Anuja : [Monday, Keirsha, Quio-Pex, Zalavali],
                                            Milio-Bubi : [Mubonada, Nulicanosi, Xiopi Kuashas]
                                            },
                            Andromeda : [Lata Nu, Kirlop, Prbhatijanu, Zua Liquor] 
    '''},
    {"page_content": "Meo Smiti was founded in 2066 by RC Pratishta."},
    {"page_content":" Meo Smiti is an MNC company that sells Arts and Artifacts."},
    {"page_content": "RC Pratishta's full name is Rhu Chandra Pratishta."}
]

In [45]:
vector_store = vectorstores.Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings.embed_query
)

vector_store.add_texts([doc["page_content"] for doc in documents])


C:\Users\Manoj\anaconda3\envs\port\lib\site-packages\langchain_community\vectorstores\pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


['2405c822-798f-4f76-939f-890f5675d7f6',
 'eea4ccf2-5f4a-46e9-9de4-ba78f2dcabe5',
 '79f35086-0c1b-483b-a606-10f0d9116f9b',
 'a54d41a2-38b4-43e5-9e22-35db4579dd88',
 'a9daa46b-a408-4efd-8e92-9539f294e114',
 '39ffa4b6-c18d-49c6-97fc-e7a548550703',
 '338baa2e-d011-4611-9d40-d2f7a78dbf07',
 '297b704d-6dfc-4612-a449-8269b0c88bf6',
 'aff2b882-01a6-4a5f-8b58-3ca4bea5eff5']

In [36]:
# Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


C:\Users\Manoj\AppData\Local\Temp\ipykernel_25696\3279284431.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [46]:
# Build conversational retrieval chain
chat_model = ChatOpenAI(model="gpt-4", openai_api_key=openai_api_key)
retrieval_chain = ConversationalRetrievalChain.from_llm(
    llm=chat_model,
    retriever=vector_store.as_retriever(),
    memory=memory
)

In [41]:
response = retrieval_chain.run("Hi")


In [42]:
response

"As an AI, I'm here to assist you. Could you please specify your question or the type of assistance you need?"

In [43]:
retrieval_chain.run("Hello")

"I'm an AI and I don't have the ability to greet you. However, I'm here to answer your questions."

In [47]:
query = "Who is JJ Kiddo "
retrieval_chain.run(query)

"JJ Kiddo is a former national-level chess player from Mandeep, Andromeda, who later became a lawyer working at Mar's BleevFlemin. He also founded the company Terrain Trait in 2089."

In [48]:
query = "Whats is his full name"
retrieval_chain.run(query)

'The full name of JJ Kiddo is Junior Juma Kiddo.'

In [49]:
query = "Who founded Meo Smiti"
retrieval_chain.run(query)

'The founder of Meo Smiti is Rhu Chandra Pratishta, also known as RC Pratishta.'

In [50]:
query = "Where is he from"
retrieval_chain.run(query)

"The text doesn't provide information on where Rhu Chandra Pratishta, the founder of Meo Smiti, is from."

In [51]:
query = "Can u give me names of persons that we just discussed about"
retrieval_chain.run(query)

'The individuals recently discussed are Junior Juma Kiddo and Rhu Chandra Pratishta (also known as RC Pratishta).'